In [1]:
from utils.read_file import *
from utils.split_law import split_law_to_token
from utils.model_tool import *

In [2]:
QUESTION_PATH = "競賽資料集/dataset/preliminary/questions_example.json"

INSURANCE_PATH = "競賽資料集/reference/insurance"
FAQ_PATH = "競賽資料集/reference/faq/pid_map_content.json"
ANS_PATH = "競賽資料集/dataset/preliminary/ground_truths_example.json"

In [3]:
question = read_json(QUESTION_PATH)["questions"]

In [4]:
# from huggingface_hub import snapshot_download

# # 下載模型到指定路徑
# local_model_path = "./models/bge-m3"
# snapshot_download(repo_id="BAAI/bge-m3", revision="main", local_dir=local_model_path)

In [5]:
from sentence_transformers import SentenceTransformer

# 加載模型
embbeded_model = SentenceTransformer("./models/bge-m3", device='cuda')

c:\Users\weiso131\anaconda3\envs\rag\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [6]:
def validate_faq(embbeded_model, question):
    faq = read_json(FAQ_PATH)
    ans = read_json(ANS_PATH)
    correct = 0
    for i in range(100, 150):
        document = read_target_faq(faq, question[i]["source"])
        k_highest, _ = get_ans(embbeded_model, question[i], document)
        predict = question[i]["source"][k_highest[0]]
        if (predict == ans["ground_truths"][i]["retrieve"]):
            correct += 1
        else:
            print(f"qid: {i + 1}, predict: {predict}, ans: {ans["ground_truths"][i]["retrieve"]}")
    return correct / 50

### 驗證faq資料集的正確率

In [7]:
validate_faq(embbeded_model, question)

C:\Users\weiso131\AppData\Roaming\Python\Python312\site-packages\transformers\models\xlm_roberta\modeling_xlm_roberta.py:371: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


qid: 109, predict: 234, ans: 283
qid: 135, predict: 399, ans: 28


0.96

In [8]:
FILE_NUM = 2000

def validate_insurance(embbeded_model, question):
    ans = read_json(ANS_PATH)
    correct = 0
    half_correct = 0 #答案在前五

    already_read = ["" for _ in range(FILE_NUM)]


    for i in range(0, 50):
        src = np.array(question[i]["source"])
        document = read_target_insurance_pdf(INSURANCE_PATH, src, already_read)
        tokens = []
        chunk_real_index = []
        for j in range(len(document)):
            text = document[j]

            if already_read[src[j]] == "":
                already_read[src[j]] = text

            chunks = split_law_to_token(text)
            tokens.extend(chunks)
            chunk_real_index.extend([j + k * 0 for k in range(len(chunks))])

                
        chunk_real_index = np.array(chunk_real_index)

        k = 10
        token_index, similarities = get_ans(embbeded_model, question[i], tokens, k)
        real_index = chunk_real_index[token_index]
        all_predict = src[real_index]
        predict = all_predict[0]


        if (predict == ans["ground_truths"][i]["retrieve"]):
            correct += 1
        else:
            show_wrong_ans(i, predict, ans, token_index, \
                   src, real_index, similarities, chunk_real_index, tokens, "output_insurance")


        if (ans["ground_truths"][i]["retrieve"] in all_predict):
            half_correct += 1

    print(f"acc: {correct / 50 * 100} %")
    print(f"in rank 5: {half_correct / 50 * 100} %")

### 驗證insurance資料集的正確率

In [9]:
import os

if not os.path.exists("output_insurance/"):
    os.makedirs("output_insurance/")

validate_insurance(embbeded_model, question)

C:\Users\weiso131\AppData\Roaming\Python\Python312\site-packages\transformers\models\xlm_roberta\modeling_xlm_roberta.py:371: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


qid: 2, predict: 258, ans: 428
qid: 4, predict: 179, ans: 186
qid: 50, predict: 555, ans: 78
acc: 94.0 %
in rank 5: 96.0 %
